# Download data

In [ ]:
import pandas as pd
import numpy as np
import os
import tqdm
import mimetypes
import requests
from requests import Session
from requests_html import AsyncHTMLSession
import urllib.parse
from io import BytesIO
import hashlib
from PIL import Image


root = '/path/to/download/directory/'
tab = pd.read_csv('BFree_viral_images.csv')
display(tab)



# attempting download from facebook
session = AsyncHTMLSession()
async def fetch_facebook_image_url(url):
    response = await session.get(url, timeout=10, allow_redirects=True)
    
    # looking for the image url in the html
    await response.html.arender(timeout=5)
    img_element = response.html.find('img.x85a59c.x193iq5w.x4fas0m.x19kjcj4', first=True)
    
    if img_element:
        url = img_element.attrs.get('src')
        print(f"Image URL: {url}")
        return url
    else:
        img_elements = response.html.find('img')
        for img_element in img_elements:
            src = img_element.attrs.get('src', '')
            if 'https://scontent.fnap6-1.fna.fbcdn.net' in src:
                print(f"Found Image URL: {src}")
                return url
        print(f"Image not found. {url}")
        
        
        
with Session() as s:
    s.headers.update({'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',})
      
    for index in tqdm.tqdm(tab.index):
        url  = tab.loc[index,'url']
        filename = os.path.join(root, tab.loc[index,'filename'])

        # if on facebook, looking for the actual url
        if url.startswith('https://lookaside.fbsbx.com'):
            try:
                url = await fetch_facebook_image_url(url)
            except:
                import traceback
                traceback.print_exc()
                continue
        # request
        try:
            response = s.get(url, timeout=10, allow_redirects=True)
        except Exception as e:
            print('\n Request error', index, url, e, flush=True)
            continue
            
        # saving file
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        try:
            with open(filename, 'wb') as fo:
                fo.write(response.content)
            
            # checking if md5sum matches (the link may now point to a "404" error image)
            md5 = hashlib.md5(open(filename,'rb').read()).hexdigest()
            if md5 != tab.loc[index, 'md5']:
                print('Warning: md5 mishatch!')
                if os.path.exists(filename):
                    os.remove(filename)
                continue
        except:
            print('\n Save error', index, extension, url, content_type, flush=True)
            continue